In [ ]:
import os
from time import sleep
# Dataset augmentation types: 
# TPG: TimeStretch + PitchShift + GaussianSNR
# TPGB: TimeStretch + PitchShift + GaussianSNR + Background Noises
# TPGBIR: TimeStretch + PitchShift + GaussianSNR + Background Noises + Impulse Responses
aug_type = "TPGBIR"

# must provide initial model state
model_state = "2025-11-12T14-29-42_0.0308"
model_choice = "openai_whisper_small"


model_base_path = os.path.join("model", model_choice)
model_state_path = os.path.join(model_base_path, model_state)

def get_latest_model_path(base_path):
    model_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    latest_model_dir = max(model_dirs, key=lambda d: os.path.getmtime(os.path.join(base_path, d)) if "checkpoint" not in d else 0)
    print(f"Latest model directory: {latest_model_dir}")
    return os.path.join(base_path, latest_model_dir)

# get_latest_model_dir(model_base_path)

i=0
while True:
    print(f"Running auto fine tune pipeline iteration {i}")
    %run preprocess.py --aug_type {aug_type} --src_dir "train" --label_file "train-toneless.csv" --dist_dir "train_{aug_type}" --n_augmentations 5
    %run preprocess.py --aug_type {aug_type} --src_dir "dict-sentence" --label_file "dict-sentence.csv" --dist_dir "dict-sentence_{aug_type}" --n_augmentations 1
    %run preprocess.py --aug_type {aug_type} --src_dir "dict-word" --label_file "dict-word" --dist_dir "dict-word_{aug_type}" --n_augmentations 1
    if i!=0:
        model_state_path = get_latest_model_path(model_base_path)
    
    %run train.py --dataset f"train_{aug_type}" --model_choice {model_choice} --model_state_path {model_state_path}
    model_state_path = get_latest_model_path(model_base_path)
    %run prediction.py --model_dir {model_state_path}

    %run train.py --dataset f"dict-sentence_{aug_type}" --model_choice {model_choice} --model_state_path {model_state_path}
    model_state_path = get_latest_model_path(model_base_path)
    %run prediction.py --model_dir {model_state_path}

    %run train.py --dataset f"train_{aug_type}" --model_choice {model_choice} --model_state_path {model_state_path}
    model_state_path = get_latest_model_path(model_base_path)
    %run prediction.py --model_dir {model_state_path}

    %run train.py --dataset f"dict-word_{aug_type}" --model_choice {model_choice} --model_state_path {model_state_path}
    model_state_path = get_latest_model_path(model_base_path)
    %run prediction.py --model_dir {model_state_path}

    i+=1
    sleep(5)
